In [ ]:
# Parameters
run_date = "2026-01-01"  # papermill replacement
import os
output_dir = os.environ.get("ORION_SIGNALS_DIR", "../signals")
config_path = os.environ.get("DATUM_API_CONFIG_PATH", "../ops/datum_api_config.json")
dry_run = False

# ensure output exists
os.makedirs(output_dir, exist_ok=True)


In [1]:
# Import basic modules
import pandas as pd
from datum_api_client import DatumApi
import datetime
from datetime import timedelta

# Import warnings
import warnings
warnings.filterwarnings("ignore")
# pip install xlrd
# pip install openpyxl

In [2]:
import os
import json
from datetime import time, datetime, date, timedelta

import numpy as np
import pandas as pd


def _add_minutes_to_time(t: time, minutes: int) -> time:
    """Допоміжна: додає minutes до time (без врахування переходу через добу)."""
    base = datetime.combine(date(2000, 1, 1), t)
    res = base + timedelta(minutes=minutes)
    return res.time()


def analyze_intraday_windows(
    input_parquet: str,
    output_dir: str,
    *,
    # назви колонок у вхідному файлі
    time_col: str = "dt",
    ticker_col: str = "ticker",
    stack_col: str = "Stack%",   # стак%
    bench_col: str = "Bench%",   # бенч%
    devsig_col: str = "dev_sig", # dev сигма
    # часові вікна: (назва, nominal_from, nominal_to)
    windows=None,
    # параметри для бінінгу
    n_bins: int = 6,
    grow_threshold: float = 0.55,  # коли grow_rate >= цього → "росте"
    fall_threshold: float = 0.45,  # коли grow_rate <= цього → "падає"
    min_total_in_bin: int = 5,     # мін. кількість спостережень у біні
) -> None:
    """
    Аналіз рухів по фіксованих вікнах часу.

    summary.csv (ГОЛОВНА ЗМІНА):
      - ОДИН рядок на тікер.
      - Колонки, які потрібні фронту:
          ticker, bench, corr, beta, sig,
          events_total,
          early_blue_up_rate, early_blue_down_rate,
          early_ark_up_rate, early_ark_down_rate,
          middle_ark_up_rate, middle_ark_down_rate,
          late_ark_up_rate, late_ark_down_rate,
          open_up_rate, open_down_rate,
          early_post_up_rate, early_post_down_rate,
          late_post_up_rate, late_post_down_rate,
        + інші технічні колонки (total_delta, mean_delta_total тощо), які можна юзати в extras.

    onefile.jsonl / best_params.json — без змін.
    """

    # --------------------------------------------------------
    # 0. Дефолтні вікна
    # --------------------------------------------------------
    if windows is None:
        windows = [
            ("EARLY_BLUE",  time(0, 0),  time(0, 30)),
            ("EARLY_ARK",   time(3, 50), time(4, 30)),
            ("MIDDLE_ARK",  time(5, 50), time(6, 30)),
            ("LATE_ARK",    time(7, 50), time(8, 30)),
            ("OPEN",        time(9, 20), time(9, 40)),
            ("EARLY_POST",  time(15, 50), time(16, 30)),
            ("LATE_POST",   time(19, 30), time(19, 59)),
        ]

    os.makedirs(output_dir, exist_ok=True)

    summary_path = os.path.join(output_dir, "summary.csv")
    onefile_path = os.path.join(output_dir, "onefile.jsonl")
    best_params_path = os.path.join(output_dir, "best_params.json")

    # --------------------------------------------------------
    # 1. Читаємо дані
    # --------------------------------------------------------
    cols = [time_col, ticker_col, stack_col, bench_col, devsig_col]
    df = pd.read_parquet(input_parquet, columns=cols)

    required_cols = cols
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"У вхідному файлі немає колонок: {missing}")

    mask_basic = df[time_col].notna() & df[ticker_col].notna()
    df = df.loc[mask_basic]

    df[time_col] = pd.to_datetime(df[time_col])
    df["session_date"] = df[time_col].dt.date
    df["time_only"] = df[time_col].dt.time

    for c in (stack_col, bench_col, devsig_col):
        df[c] = pd.to_numeric(df[c], errors="coerce")

    mask_num = df[stack_col].notna() & df[bench_col].notna() & df[devsig_col].notna()
    df = df.loc[mask_num].reset_index(drop=True)

    # --------------------------------------------------------
    # 2. Конструюємо events по кожному вікну
    # --------------------------------------------------------
    events_frames = []

    for win_name, t_from, t_to in windows:
        start_to = _add_minutes_to_time(t_from, 10)
        end_to = _add_minutes_to_time(t_to, 10)

        mask_window = (df["time_only"] >= t_from) & (df["time_only"] <= end_to)
        wdf = df.loc[mask_window].copy()
        if wdf.empty:
            continue

        # START
        start_mask = (wdf["time_only"] >= t_from) & (wdf["time_only"] <= start_to)
        w_start = wdf.loc[start_mask]
        if w_start.empty:
            continue
        w_start = w_start.sort_values(time_col)
        start_idx = w_start.groupby([ticker_col, "session_date"])[time_col].idxmin()
        start_rows = w_start.loc[start_idx]

        # END
        end_mask = (wdf["time_only"] >= t_to) & (wdf["time_only"] <= end_to)
        w_end = wdf.loc[end_mask]
        if w_end.empty:
            continue
        w_end = w_end.sort_values(time_col)
        end_idx = w_end.groupby([ticker_col, "session_date"])[time_col].idxmin()
        end_rows = w_end.loc[end_idx]

        # merge start/end
        start_rows = start_rows.reset_index(drop=True)
        end_rows = end_rows.reset_index(drop=True)

        merged = pd.merge(
            start_rows,
            end_rows,
            on=[ticker_col, "session_date"],
            suffixes=("_start", "_end"),
            how="inner",
        )
        if merged.empty:
            continue

        start_dt_col = f"{time_col}_start"
        end_dt_col = f"{time_col}_end"
        merged = merged[merged[end_dt_col] > merged[start_dt_col]]
        if merged.empty:
            continue

        events_win = pd.DataFrame({
            ticker_col: merged[ticker_col],
            "session_date": merged["session_date"],
            "window": win_name,
            "start_dt": merged[start_dt_col],
            "end_dt": merged[end_dt_col],
            "stack_start": merged[f"{stack_col}_start"].astype(float),
            "stack_end": merged[f"{stack_col}_end"].astype(float),
            "bench_start": merged[f"{bench_col}_start"].astype(float),
            "bench_end": merged[f"{bench_col}_end"].astype(float),
            "dev_start": merged[f"{devsig_col}_start"].astype(float),
            "dev_end": merged[f"{devsig_col}_end"].astype(float),
        })

        events_win["delta_stack"] = events_win["stack_end"] - events_win["stack_start"]
        events_win["delta_bench"] = events_win["bench_end"] - events_win["bench_start"]
        events_win["delta_dev"] = events_win["dev_end"] - events_win["dev_start"]
        events_win["grew"] = events_win["delta_stack"] > 0

        events_frames.append(events_win)

    if not events_frames:
        raise ValueError(
            "Немає жодної події в жодному вікні (навіть з +10 хв толерансом). "
            "Перевір вхідний файл / таймзону / вікна."
        )

    events = pd.concat(events_frames, ignore_index=True)

    # --------------------------------------------------------
    # 3. summary_df по (ticker, window) — проміжний крок
    # --------------------------------------------------------
    g_tw = events.groupby([ticker_col, "window"])

    agg_base = (
        g_tw
        .agg(
            n=("delta_stack", "count"),
            total_delta=("delta_stack", "sum"),
            mean_delta=("delta_stack", "mean"),
            median_delta=("delta_stack", "median"),
        )
        .reset_index()
    )

    up_mask = events["delta_stack"] > 0
    down_mask = events["delta_stack"] < 0

    up_stats = (
        events.loc[up_mask]
        .groupby([ticker_col, "window"])
        .agg(
            n_up=("delta_stack", "count"),
            mean_up=("delta_stack", "mean"),
            median_up=("delta_stack", "median"),
        )
        .reset_index()
    )

    down_stats = (
        events.loc[down_mask]
        .groupby([ticker_col, "window"])
        .agg(
            n_down=("delta_stack", "count"),
            mean_down=("delta_stack", "mean"),
            median_down=("delta_stack", "median"),
        )
        .reset_index()
    )

    summary_df = (
        agg_base
        .merge(up_stats, on=[ticker_col, "window"], how="left")
        .merge(down_stats, on=[ticker_col, "window"], how="left")
    )

    summary_df["n_up"] = summary_df["n_up"].fillna(0).astype(int)
    summary_df["n_down"] = summary_df["n_down"].fillna(0).astype(int)

    summary_df["mean_up"] = summary_df["mean_up"].where(summary_df["n_up"] > 0, np.nan)
    summary_df["median_up"] = summary_df["median_up"].where(summary_df["n_up"] > 0, np.nan)
    summary_df["mean_down"] = summary_df["mean_down"].where(summary_df["n_down"] > 0, np.nan)
    summary_df["median_down"] = summary_df["median_down"].where(summary_df["n_down"] > 0, np.nan)

    summary_df["n_flat"] = summary_df["n"] - summary_df["n_up"] - summary_df["n_down"]
    summary_df["up_ratio"] = summary_df["n_up"] / summary_df["n"]
    summary_df["down_ratio"] = summary_df["n_down"] / summary_df["n"]

    summary_df = summary_df[
        [
            ticker_col,
            "window",
            "n",
            "n_up",
            "n_down",
            "n_flat",
            "up_ratio",
            "down_ratio",
            "total_delta",
            "mean_delta",
            "median_delta",
            "mean_up",
            "median_up",
            "mean_down",
            "median_down",
        ]
    ].sort_values([ticker_col, "window"])

    # --------------------------------------------------------
    # 3b. Перетворюємо summary_df → ОДИН РЯДОК НА ТІКЕР (ПІД ФРОНТ)
    # --------------------------------------------------------
    # Глобальні метрики по тікеру (агрегація по всіх вікнах)
    g_t = events.groupby(ticker_col)

    global_df = (
        g_t
        .agg(
            events_total=("delta_stack", "count"),
            total_delta=("delta_stack", "sum"),
            mean_delta_total=("delta_stack", "mean"),
            median_delta_total=("delta_stack", "median"),
        )
        .reset_index()
    )

    # up/down/flat по всіх вікнах
    up_global = (
        events.loc[events["delta_stack"] > 0]
        .groupby(ticker_col)
        .agg(n_up_total=("delta_stack", "count"))
        .reset_index()
    )
    down_global = (
        events.loc[events["delta_stack"] < 0]
        .groupby(ticker_col)
        .agg(n_down_total=("delta_stack", "count"))
        .reset_index()
    )

    global_df = (
        global_df
        .merge(up_global, on=ticker_col, how="left")
        .merge(down_global, on=ticker_col, how="left")
    )

    global_df["n_up_total"] = global_df["n_up_total"].fillna(0).astype(int)
    global_df["n_down_total"] = global_df["n_down_total"].fillna(0).astype(int)
    global_df["n_flat_total"] = (
        global_df["events_total"] - global_df["n_up_total"] - global_df["n_down_total"]
    ).astype(int)

    global_df["up_ratio_total"] = global_df["n_up_total"] / global_df["events_total"]
    global_df["down_ratio_total"] = global_df["n_down_total"] / global_df["events_total"]

    # wide-таблиця по вікнах: (metric, window) → колонки
    wide = summary_df.pivot(
        index=ticker_col,
        columns="window",
        values=[
            "n",
            "n_up",
            "n_down",
            "n_flat",
            "up_ratio",
            "down_ratio",
            "total_delta",
            "mean_delta",
            "median_delta",
            "mean_up",
            "median_up",
            "mean_down",
            "median_down",
        ],
    )

    # (metric, window) → f"{window}_{metric}", напр. EARLY_ARK_up_ratio
    wide.columns = [
        f"{win}_{metric}"
        for (metric, win) in wide.columns.to_flat_index()
    ]
    wide = wide.reset_index()

    # мерджимо глобальні колонки + wide по вікнах
    summary_final = (
        global_df
        .merge(wide, on=ticker_col, how="left")
        .sort_values(ticker_col)
    )

    # --------------------------------------------------------
    # 3c. Додаємо meta-колонки bench/corr/beta/sig (порожні, якщо немає)
    # --------------------------------------------------------
    for c in ["bench", "corr", "beta", "sig"]:
        if c not in summary_final.columns:
            summary_final[c] = ""

    # --------------------------------------------------------
    # 3d. Аліаси під фронт: *_up_rate / *_down_rate
    #     Беремо з <WINDOW>_up_ratio / <WINDOW>_down_ratio
    # --------------------------------------------------------
    def alias_rate(window_name: str, alias_prefix: str):
        up_src = f"{window_name}_up_ratio"
        dn_src = f"{window_name}_down_ratio"

        up_alias = f"{alias_prefix}_up_rate"
        dn_alias = f"{alias_prefix}_down_rate"

        if up_src in summary_final.columns:
            summary_final[up_alias] = summary_final[up_src]
        else:
            summary_final[up_alias] = np.nan

        if dn_src in summary_final.columns:
            summary_final[dn_alias] = summary_final[dn_src]
        else:
            summary_final[dn_alias] = np.nan

    alias_rate("EARLY_BLUE",  "early_blue")
    alias_rate("EARLY_ARK",   "early_ark")
    alias_rate("MIDDLE_ARK",  "middle_ark")
    alias_rate("LATE_ARK",    "late_ark")
    alias_rate("OPEN",        "open")
    alias_rate("EARLY_POST",  "early_post")
    alias_rate("LATE_POST",   "late_post")

    # --------------------------------------------------------
    # 3e. Порядок колонок: спочатку те, що юзає фронт
    # --------------------------------------------------------
    preferred_order = [
        ticker_col,
        "bench",
        "corr",
        "beta",
        "sig",
        "events_total",
        "total_delta",
        "mean_delta_total",
        "median_delta_total",
        "n_up_total",
        "n_down_total",
        "n_flat_total",
        "up_ratio_total",
        "down_ratio_total",

        "early_blue_up_rate",
        "early_blue_down_rate",
        "early_ark_up_rate",
        "early_ark_down_rate",
        "middle_ark_up_rate",
        "middle_ark_down_rate",
        "late_ark_up_rate",
        "late_ark_down_rate",
        "open_up_rate",
        "open_down_rate",
        "early_post_up_rate",
        "early_post_down_rate",
        "late_post_up_rate",
        "late_post_down_rate",
    ]

    other_cols = [c for c in summary_final.columns if c not in preferred_order]
    summary_final = summary_final[preferred_order + other_cols]

    # фінальний запис CSV
    summary_final.to_csv(summary_path, index=False)

    # мапа (ticker, window) → рядок summary_df (для onefile.jsonl)
    summary_map = {
        (row[ticker_col], row["window"]): row
        for _, row in summary_df.iterrows()
    }

    # --------------------------------------------------------
    # 4. Хелпери для бінінгу та examples (ONEFILE / BEST_PARAMS)
    # --------------------------------------------------------
    def build_bins_for_feature(df_group: pd.DataFrame, feature_col: str):
        series = df_group[feature_col].dropna()
        if len(series) < min_total_in_bin:
            return []

        try:
            labels = pd.qcut(series, q=min(n_bins, series.nunique()), duplicates="drop")
        except Exception:
            return []

        tmp = df_group.copy()
        tmp["bin"] = labels

        res = []
        for b, gb in tmp.groupby("bin"):
            n = len(gb)
            if n < min_total_in_bin:
                continue

            delta = gb["delta_stack"]
            pos = delta[delta > 0]
            neg = delta[delta < 0]
            grow_rate = float((delta > 0).mean()) if n > 0 else 0.0

            left = float(b.left)
            right = float(b.right)

            res.append(
                {
                    "from": left,
                    "to": right,
                    "n": int(n),
                    "grow_rate": grow_rate,
                    "mean_delta": float(delta.mean()),
                    "median_delta": float(delta.median()),
                    "mean_up": float(pos.mean()) if len(pos) else None,
                    "median_up": float(pos.median()) if len(pos) else None,
                    "mean_down": float(neg.mean()) if len(neg) else None,
                    "median_down": float(neg.median()) if len(neg) else None,
                }
            )
        return res

    def build_bins_2d(df_group: pd.DataFrame, x_col: str, y_col: str):
        df_xy = df_group[[x_col, y_col, "delta_stack"]].dropna()
        if len(df_xy) < min_total_in_bin:
            return []

        x = df_xy[x_col]
        y = df_xy[y_col]

        try:
            x_bins = pd.qcut(x, q=min(n_bins, x.nunique()), duplicates="drop")
            y_bins = pd.qcut(y, q=min(n_bins, y.nunique()), duplicates="drop")
        except Exception:
            return []

        tmp = df_xy.copy()
        tmp["x_bin"] = x_bins
        tmp["y_bin"] = y_bins

        res = []
        for (bx, by), gb in tmp.groupby(["x_bin", "y_bin"]):
            n = len(gb)
            if n < min_total_in_bin:
                continue

            delta = gb["delta_stack"]
            pos = delta[delta > 0]
            neg = delta[delta < 0]
            grow_rate = float((delta > 0).mean()) if n > 0 else 0.0

            res.append(
                {
                    "dev_from": float(bx.left),
                    "dev_to": float(bx.right),
                    "bench_from": float(by.left),
                    "bench_to": float(by.right),
                    "n": int(n),
                    "grow_rate": grow_rate,
                    "mean_delta": float(delta.mean()),
                    "median_delta": float(delta.median()),
                    "mean_up": float(pos.mean()) if len(pos) else None,
                    "median_up": float(pos.median()) if len(pos) else None,
                    "mean_down": float(neg.mean()) if len(neg) else None,
                    "median_down": float(neg.median()) if len(neg) else None,
                }
            )
        return res

    def build_examples(df_tw: pd.DataFrame, kind: str, limit: int = 3):
        if kind == "up":
            mask = df_tw["delta_stack"] > 0
        elif kind == "down":
            mask = df_tw["delta_stack"] < 0
        else:
            return []

        df_k = df_tw.loc[mask]
        if df_k.empty:
            return []

        df_k = df_k.sort_values(["session_date", "end_dt"])
        df_last = df_k.tail(limit)

        examples = []
        for _, row in df_last.iterrows():
            examples.append(
                {
                    "date": str(row["session_date"]),
                    "start_dt": row["start_dt"].isoformat(),
                    "end_dt": row["end_dt"].isoformat(),
                    "stack_start": float(row["stack_start"]),
                    "stack_end": float(row["stack_end"]),
                    "bench_start": float(row["bench_start"]),
                    "bench_end": float(row["bench_end"]),
                    "dev_start": float(row["dev_start"]),
                    "dev_end": float(row["dev_end"]),
                    "delta_stack": float(row["delta_stack"]),
                    "delta_bench": float(row["delta_bench"]),
                    "delta_dev": float(row["delta_dev"]),
                }
            )
        return examples

    # --------------------------------------------------------
    # 5. onefile.jsonl + best_params.json (без змін)
    # --------------------------------------------------------
    best_params: dict[str, dict] = {}
    onefile_out = open(onefile_path, "w", encoding="utf-8")

    for ticker, df_ticker in events.groupby(ticker_col):

        ticker_record = {
            "ticker": ticker,
            "windows": {}
        }

        best_params[ticker] = {}

        for win_name, df_tw in df_ticker.groupby("window"):
            stats_row = summary_map.get((ticker, win_name))
            if stats_row is None:
                continue

            # 1D-біни
            stack_bins = build_bins_for_feature(df_tw, "stack_start")
            bench_bins = build_bins_for_feature(df_tw, "bench_start")
            dev_bins = build_bins_for_feature(df_tw, "dev_start")

            bins_1d = {
                "stack_start": stack_bins,
                "bench_start": bench_bins,
                "dev_start": dev_bins,
            }

            # 2D-біни dev_start vs bench_start
            bins2d_dev_bench = build_bins_2d(df_tw, "dev_start", "bench_start")

            # приклади
            examples_up = build_examples(df_tw, "up", limit=3)
            examples_down = build_examples(df_tw, "down", limit=3)

            ticker_record["windows"][win_name] = {
                "stats": {
                    "n": int(stats_row["n"]),
                    "n_up": int(stats_row["n_up"]),
                    "n_down": int(stats_row["n_down"]),
                    "n_flat": int(stats_row["n_flat"]),
                    "up_ratio": float(stats_row["up_ratio"]),
                    "down_ratio": float(stats_row["down_ratio"]),
                    "total_delta": float(stats_row["total_delta"]),
                    "mean_delta": float(stats_row["mean_delta"]),
                    "median_delta": float(stats_row["median_delta"]),
                    "mean_up": None if pd.isna(stats_row["mean_up"]) else float(stats_row["mean_up"]),
                    "median_up": None if pd.isna(stats_row["median_up"]) else float(stats_row["median_up"]),
                    "mean_down": None if pd.isna(stats_row["mean_down"]) else float(stats_row["mean_down"]),
                    "median_down": None if pd.isna(stats_row["median_down"]) else float(stats_row["median_down"]),
                },
                "bins_1d": bins_1d,
                "bins_2d": {
                    "dev_vs_bench": bins2d_dev_bench
                },
                "examples": {
                    "up": examples_up,
                    "down": examples_down,
                },
            }

            # best_params з 1D-бінів
            win_best = {}

            def merge_bins(bins_list):
                if not bins_list:
                    return []

                bins_sorted = sorted(bins_list, key=lambda x: x["from"])
                merged = []
                cur = {
                    "from": bins_sorted[0]["from"],
                    "to": bins_sorted[0]["to"],
                    "total": bins_sorted[0]["n"],
                    "sum_rate": bins_sorted[0]["grow_rate"] * bins_sorted[0]["n"],
                }

                for b in bins_sorted[1:]:
                    if b["from"] <= cur["to"]:
                        cur["to"] = max(cur["to"], b["to"])
                        cur["total"] += b["n"]
                        cur["sum_rate"] += b["grow_rate"] * b["n"]
                    else:
                        rate = cur["sum_rate"] / cur["total"] if cur["total"] > 0 else 0.0
                        merged.append(
                            {
                                "from": float(cur["from"]),
                                "to": float(cur["to"]),
                                "rate": float(rate),
                                "total": int(cur["total"]),
                            }
                        )
                        cur = {
                            "from": b["from"],
                            "to": b["to"],
                            "total": b["n"],
                            "sum_rate": b["grow_rate"] * b["n"],
                        }

                rate = cur["sum_rate"] / cur["total"] if cur["total"] > 0 else 0.0
                merged.append(
                    {
                        "from": float(cur["from"]),
                        "to": float(cur["to"]),
                        "rate": float(rate),
                        "total": int(cur["total"]),
                    }
                )
                return merged

            for feat_name, feat_bins in bins_1d.items():
                if not feat_bins:
                    continue

                up_bins = [
                    b for b in feat_bins
                    if (b["grow_rate"] >= grow_threshold and b["n"] >= min_total_in_bin)
                ]
                down_bins = [
                    b for b in feat_bins
                    if (b["grow_rate"] <= fall_threshold and b["n"] >= min_total_in_bin)
                ]

                win_best[feat_name] = {
                    "up": merge_bins(up_bins),
                    "down": merge_bins(down_bins),
                }

            best_params[ticker][win_name] = win_best

        onefile_out.write(json.dumps(ticker_record, ensure_ascii=False) + "\n")

    onefile_out.close()

    # --------------------------------------------------------
    # 6. best_params.json
    # --------------------------------------------------------
    with open(best_params_path, "w", encoding="utf-8") as f:
        json.dump(best_params, f, ensure_ascii=False, indent=2)

    print("Готово:")
    print(f"  summary     -> {summary_path}")
    print(f"  onefile     -> {onefile_path}")
    print(f"  best_params -> {best_params_path}")


In [ ]:
# analyze_intraday_windows(
#     input_parquet="ARBITRAGE/final_filtered.parquet",
#     output_dir="ARBITRAGE/windows_analysis",
#     n_bins=6,
#     grow_threshold=0.55,
#     fall_threshold=0.45,
#     min_total_in_bin=5,
# )


Готово:
  summary     -> ARBITRAGE/windows_analysis\summary.csv
  onefile     -> ARBITRAGE/windows_analysis\onefile.jsonl
  best_params -> ARBITRAGE/windows_analysis\best_params.json
